In [9]:
%pip install -qU langchain-google-genai langchain-huggingface sentence_transformers hf_xet langchain_postgres langchain_community pandas ipywidgets tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from IPython.display import Markdown
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_postgres import PGVector
from dotenv import load_dotenv
import os

In [11]:
load_dotenv()

# Using local embeddings model downloaded from Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

connection = os.getenv("POSTGRES_URL")
collection_name = "csv_docs"

csv_vector_store = PGVector(
    embeddings=embeddings,
    collection_name=collection_name,
    connection=connection,
    use_jsonb=True,
)

In [12]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.notebook import tqdm
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)

document_df = pd.read_csv("../datasets/documents.csv")

for row in tqdm(document_df.itertuples(index=False)):
    #print(len(row.text))
    text_docs = text_splitter.create_documents([row.text])
    #print(len(text_docs))
    for doc in text_docs:
        #print(doc)
        csv_vector_store.add_documents([doc])


0it [00:00, ?it/s]

In [13]:
csv_vector_store.similarity_search_with_score(query="What data did was used to test the prototype?", k=3)

[(Document(id='58e46262-3855-491e-8a49-1d53dd1085d4', metadata={}, page_content='For the prototype, I decided to make very basic configurations for everything to develop it quickly, and I tested it by asking a question about Grace Hopper, one of the most influential programmers to have ever lived, in two versions of the prototype: one with Grace Hopper’s Wikipedia page in its vector DB and one with Alan Turing’s Wikipedia page in its vector DB.\n\nThe question was:'),
  0.46894436704448883),
 (Document(id='a2034fb1-1c21-4f40-965b-62914d24f3d4', metadata={}, page_content='I will likely test out more models than this while testing out different configurations for the tool, but for the prototype, this is enough.\n\nA Model Without a Framework is Like a Car Engine Without a Chassis'),
  0.5775846335392724),
 (Document(id='d99416df-0d85-425d-87cc-9b8dd90e2cb6', metadata={}, page_content='Real world testing can only be conducted given specific safeguards, e.g. users of the systems under real